# Lecture 4: Linear Classifiers

#### Reminder from Last time

- In ML estimation we want ot max the lieklihood of having seen the dataset
- $L = p({x^1, y^1}, … {x^n, y^n})$
- Other cost functions:
  - MAP esimation: where we place a prior distribution on our parameters
  - KL divergence: We think of our scores from our model as a distribution, and the true labels as a distribution, and minimize that
  - Maximize an approximate of the data generating distribution
- Usually we have cost functions that we can't get closed form parameters for, so we use gradient descent to minimize instead



#### Image Classification

- Important computer vision task
- Viewpoint variation, bounding boxes, illumination, etc are challenging problems
- Data driven approach: train: data -> model params, test: model -> prediction



#### KNN approach

- Pick a distance metric $d(x, y)$
- choose K, the number of nearest neighbors
  - usually done w/cross validation in practice
- Go through the dataset and get k closest neighbors, use majority for voting.
- pros: const time training, simple. Cons: memory, predict time
- Infeasible for anything other than very small datasets if we have to do fast prediction
- hyperparametrs are k and the distance metric. 
- For small k, it's easy to overfit as the prediction is prone to outliers. Overfitting reduces as k increases (at the cost of bias)



#### Broadcasting with Numpy arrays

- Never really understood how this worked until now, but it makes so much sense
- Suppose you have an $N * 2$ matrix $X$ and a $2$ element array $Y$. Mathematically, you can't really do $X - y$ since their shapes don't match up
- But numpy has this feature called broadcasting, where basicaly it'll understand that what you wanted to do was subtract each row of $X$ with the quantity $Y$. Such as: 
 $$ \begin{bmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{bmatrix} - \begin{bmatrix} 1 & 2 \end{bmatrix} = \begin{bmatrix} 0 & 0 \\ 2 & 2 \\ 4 & 4 \\ \end{bmatrix} $$



#### Curse of Dimensionality

- In higher dims, the notion of distance becomes less intuitive
- volume exponentially increases as we increase dimensions. 
  - points grow further apart
  - distances in  one direction may mean more than distances in other directions.
- kNN also just looks at total distances. So you can have an image that looks visually the same, but just shift it a little bit and it will have high errors compared with the original image
  - doesn't line up to our intuition that we want our classifiers to be able to learn features that represent certain classes.

#### Linear Classifier

- Main idea: develop a score function that maps inputs to scores for each class, and assign a label based on which class gets the highest score. 
- For CIFAR-10, our linear classifier could be $y = Wx + b$ where $x \in R^{3072}$, $y \in R^{10}$, and $W\in R^{10 * 3072}$. 
- Each row of $W$ can be thought of as a template, or a separate classifier (i.e. row 0 is the "dog" template/classifier, row 9 is the "ship" classifier/template)
  ![test img](cifar.png)
  

#### What is a linear classifier doing? 
- Remember that we can write $W_1^Tx = ||W_1||||x||cos\theta$. If we normalize all of our weight vectors so that $||W|| = 1$, then we have $W_1^Tx = ||x||cos\theta$. This is basically a projection of the vector x onto the hyperplane defined by $W$. Everything on that line will have the same value for $W_1^Tx$. $W_1 is perpendicular to that line, and everything in that direction can be positively classified, and everything in the other direction is negatively classifed. See below

![test img](http://cs231n.github.io/assets/pixelspace.jpeg)



#### Softmax Function
- $softmax_i(x) = \frac{e^{a_i(x)}}{\sum_{j=1}^{C} e^{a_j}(x)}$
- We have $a_i(x) = W_i^T + b_i$. $c$ is the number of classes, $i$ is the softmax value for the $i$th score, and the numerator sums across all of the exponentiated scores. 
- Prediction with softmax: $p(y^j = i | x^j, \theta) = softmax_i(x^j)$. This means that the probability that $y^j = i$ given the $j$th training example $x^j$ and some setting of our parameters $\theta = [w_1 ... w_n]$ is given by the softmax function. 

#### Learning w/Linear Classifier
- Choose the parems $\theta = (W, b)$ thtat maxes the likelihood of having seen the data. Assume that the m training examples are iid. 
- We have a probability distribution $p(x^1 ... x^m, y^1 ... y^m \vert \theta)$, and we can write this out as $p(x^1 \vert \theta)...p(x^m \vert \theta) ... p(y^m \vert \theta) = p(x^1, y^1 \vert \theta) p(x^2, y^2 \vert \theta) ... p(x^m, y^m \vert \theta) = \prod_{i1=1}^{M} p(x^i, y^i \vert \theta)$.
- We can expand that to $\prod_{i=1}^{M} p(y^i \vert x^i, \theta)p(x^i \vert \theta)$. 
- ** Importantly **, $p(x^i \vert \theta)$ is constant with respect to our model's parameters $\theta$, because the distribution is $p(x)$ and that doesn't change if we make our parameters whatever we want to be, because the data is the data. 
- We now have $L(\theta) = \prod_{i=1}^{M} p(y^i \vert x^i \theta)$. 
- since we have $p(y^j = i | x^j, \theta) = softmax_i(x^j)$, we can substitute that into the above, take the log-likelihood, invert the signs to minimize, and simplify:
- $argmin_\theta \sum_{i=1}^{M} (log \sum_{J=1}^{C} e^{a_j(x_i)}) - a_{y^i}(x^i)$. 
- Inside the outer sum, the first expression is the sum of all of the normalized class scores. The second expression is the negative of the un-normalized score of teh correct class, so it makes sense that we awnt to min this overall quantity.
- We learn these parameters with gradient descent (next lecture!)

#### Intuition behind softmax classifier
- We have $\log p(y = i \vert x) = \log softmax_i(x) = a_i(x) - \log \sum_{i=1}^{c} \exp(a_j(x))$. 
- When we maximize this term, then if $a_i(x)$ produces the largest score, then the log-likelihood is approximately 0 (it's never going to be perfectly 0, unless we have the scores as infinity for the correct label and - infinity for the wrong labels, which can't really happen). 
- Otherwise if some other $a_j(x), j \neq i$ is the largest score, then the log-likelihood will be negative.
- So we want this term to be as close to 0 as possible (it can never go above 0). 
- Therefore, we can approximate $\log \sum_{i=1}^{c} \exp(a_j(x))$ as $max_j a_j(x)$. 
- Be careful about overflow when implementing softmax!


#### Brief overview of SVM and Hinge Loss
- maximum margin classifier: 
![test img](svm.png)

- Hinge loss function: $max(0, 1 - y^i(w^Tx^i + b)$. This basically means that we want our predictions/scores $w^Tx^i + b$ to be correct and confident. If we have our prediction as 0.2 and our label as 1, for example, we'll still incur a loss of 0.2. If we have our prediction as -0.5 and our label as 1, we'll incur a large loss of 1.5. If we have a correct and confident prediction like 1.5 then we'll incur no loss. 
- We can extend the hinge loss to multi-class classification: 
- $ H = \sum_{j \neq y^i}^{c} max(0, 1 + a_j(x^i) - a_{y^i}(x^i))$. 
- This means that we sum across our incorrect classes and compute the inner quantity. We'll incur no loss when our correct class prediction is more than our incorrect class predictions (all of them) by some margin, in this case $1$. When correct classes are equal to incorrect class sscores, our loss is $\sum_{j \neq y^i} 1 = c - 1$, so our loss is bounded by $0$ and $c-1$ when our correct class scores are more than all of our incorrect class scores. Otherwise our loss could be more than this. 
- Similar to binary SVM, where we not only penalize incorrect predictions, but also correct predictions that don't exceed some margin/threshold and are thus not "confident" enough. 